# Migration script

Migrate the input databases from the 2.29 format to the 2.31.1 format.

This notebook assumes `SCENARIO_TO_MIGRATE` points to a scenario of Kleinalbis (which buildings?) in v2.29.0 format and `EXAMPLE_SCENARIO` to a scenario in v2.31.1 format. 

These are the v2.29.0 format files to migrate:

- shapefiles 
  - inputs/building-geometry/zone.shp... checked. no changes!
  - inputs/building-geometry/site.shp... checked. no changes!
  - inputs/building-geometry/surroundings.shp... checked. no changes!
- inputs to archetypes-mapper
  - inputs/building-properties/age.dbf
  - inputs/building-properties/occupancy.dbf
- outputs of archety
- inputs/building-properties/air_conditioning_systems.dbf
- inputs/building-properties/architecture.dbf
- inputs/building-properties/indoor_comfort.dbf
- inputs/building-properties/internal_loads.dbf
- inputs/building-properties/supply_systems.dbf
- inputs/building-properties/schedules/B*.csv

Also, remember, the inputs/technology folder also changed and the v2.31.1 format references these possibly differently.

In [5]:
# Constants
SCENARIO_TO_MIGRATE = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\2.29.0\kleinalbis"
EXAMPLE_SCENARIO = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\working-with-databases\kleinalbis"

In [22]:
import os
import cea
import geopandas

import cea.inputlocator
from cea.utilities.dbf import dbf_to_dataframe

In [11]:
# we can't use the input locator for the v2.29.0 scenario, but we _can_ (and should) use it for v2.31.1.
locator = cea.inputlocator.InputLocator(scenario=EXAMPLE_SCENARIO)

## Migrate building-geometry files

In [13]:
# let's first compare the two files (and practice reading/writing shape files)
old_zone_shp = geopandas.GeoDataFrame.from_file(
    os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-geometry", "zone.shp"))
old_zone_shp

,Name,height_ag,floors_ag,height_bg,floors_bg,descriptio,category,REFERENCE,geometry
0,B1000,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."
1,B1001,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462736.5184668251 15245045.1003448, ..."
2,B1002,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462730.7804993654 15245054.87234257,..."
3,B1003,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462743.7118557409 15245049.30022802,..."
4,B1004,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462737.9663290229 15245059.07226665,..."
5,B1005,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."


In [14]:
new_zone_shp = geopandas.GeoDataFrame.from_file(locator.get_zone_geometry())
new_zone_shp

,Name,height_ag,floors_ag,height_bg,floors_bg,descriptio,category,REFERENCE,geometry
0,B1000,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."
1,B1001,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462736.5184668251 15245045.1003448, ..."
2,B1002,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462730.7804993654 15245054.87234257,..."
3,B1003,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462743.7118557409 15245049.30022802,..."
4,B1004,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462737.9663290229 15245059.07226665,..."
5,B1005,9.0,3,3.0,1,FGZ-Siedlung 12: Kleinalbis II,house,OSM - as it is,"POLYGON ((462732.2208216225 15245068.84431113,..."


The zone.shp file is the the same! (repeated this procedure with site.shp and surroundings.shp)

## Migrate age.dbf file

In [23]:
old_age_dbf = dbf_to_dataframe(os.path.join(SCENARIO_TO_MIGRATE, "inputs", "building-properties", "age.dbf"))
old_age_dbf

,HVAC,Name,REFERENCE,basement,built,envelope,partitions,roof,windows
0,0,B1000,OSM - as it is,0,1943,0,0,0,0
1,0,B1001,OSM - as it is,0,1943,0,0,0,0
2,0,B1002,OSM - as it is,0,1943,0,0,0,0
3,0,B1003,OSM - as it is,0,1943,0,0,0,0
4,0,B1004,OSM - as it is,0,1943,0,0,0,0
5,0,B1005,OSM - as it is,0,1943,0,0,0,0


In [26]:
# i think that should be migrated to typology.dbf
new_typology_dbf = dbf_to_dataframe(locator.get_building_typology())
new_typology_dbf

,1ST_USE,1ST_USE_R,2ND_USE,2ND_USE_R,3RD_USE,3RD_USE_R,Name,REFERENCE,STANDARD,YEAR
0,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1000,OSM - as it is,STANDARD2,1943
1,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1001,OSM - as it is,STANDARD2,1943
2,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1002,OSM - as it is,STANDARD2,1943
3,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1003,OSM - as it is,STANDARD2,1943
4,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1004,OSM - as it is,STANDARD2,1943
5,MULTI_RES,1.0,NONE,0.0,NONE,0.0,B1005,OSM - as it is,STANDARD2,1943


OK. So. This is going to be interesting.

- old_age_dbf.Name --> new_typology_dbf.Name
- old_age_dbf.built --> new_typology_dbf.YEAR

(I'm assuming we just forget the other stuff for now - maybe when we look into the old technologies folder, we'll figure out a way to translate the HVAC, basement, envelope, partitions, roof and windows fields. Maybe.)

In [ ]:
 ## Migrate occupancy.dbf file